<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/RUG_QnA_MODEL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RAG model using BERT and LangChain as the retriever and BLOOM as the generator.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=3a78d4c31ac61fb30819f89a3d1155ee924c51a54ac45ac5c09b443e467d4bb3
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [3]:
!pip install python-docx
from docx import Document


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.5 MB/s eta 0:00:00


In [7]:
from docx import Document

# Load the document
doc_path = "/content/sample_data/Hair Document.docx"  # Update the path if necessary


In [8]:
doc = Document(doc_path)

In [9]:
# Extract text from the document
doc_text = ""
for paragraph in doc.paragraphs:
    doc_text += paragraph.text + "\n"

In [10]:
# Display the first few characters of the document text
print(doc_text[:200])

Hair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual. Significant progress is being made on discovering an effectiv
